In [36]:
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
# from tensorflow.keras.optimizers import legacy
from sklearn.metrics import classification_report, confusion_matrix

In [37]:
dataset_dir = 'private/tomatect'
base_dir = 'private/splits'
train_pct = 0.8
validation_pct = 0.15
test_pct = 0.05
img_width = 150
img_height = 150
batch_size = 16

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [38]:
dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=123,
    shuffle=True,
    image_size=(img_height,img_width),
    batch_size=batch_size
)

dataset_validation = tf.keras.preprocessing.image_dataset_from_directory(
    validation_dir,
    seed=123,
    shuffle=True,
    image_size=(img_height,img_width),
    batch_size=batch_size
)


dataset_test = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=123,
    shuffle=False,
    image_size=(img_height,img_width),
    batch_size=batch_size
)

Found 183 files belonging to 5 classes.
Found 30 files belonging to 5 classes.
Found 9 files belonging to 5 classes.


In [39]:
class_names = dataset_train.class_names

dataset_train = dataset_train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
dataset_validation = dataset_validation.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
dataset_test = dataset_test.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [40]:
data_scaling = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(img_height, img_width),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
   tf.keras.layers.experimental.preprocessing.RandomRotation(0.9),
   tf.keras.layers.experimental.preprocessing.RandomContrast(0.5),
  tf.keras.layers.experimental.preprocessing.RandomZoom(0.2), 
])

In [41]:
train_ds = dataset_train.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [42]:
input_shape = (batch_size, img_width, img_height, 3)
n_classes = 5

In [57]:
model = tf.keras.models.Sequential([
    data_scaling,
  #  data_augmentation,
    tf.keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(n_classes, activation='softmax'),
])

In [59]:
model.build(input_shape=input_shape)

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_6 (Sequential)   (None, 150, 150, 3)       0         
                                                                 
 conv2d_33 (Conv2D)          (16, 148, 148, 32)        896       
                                                                 
 max_pooling2d_33 (MaxPooli  (16, 74, 74, 32)          0         
 ng2D)                                                           
                                                                 
 conv2d_34 (Conv2D)          (16, 72, 72, 64)          18496     
                                                                 
 max_pooling2d_34 (MaxPooli  (16, 36, 36, 64)          0         
 ng2D)                                                           
                                                                 
 conv2d_35 (Conv2D)          (16, 34, 34, 64)        

In [60]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [66]:
dataset_validation.shape

AttributeError: '_PrefetchDataset' object has no attribute 'shape'

In [61]:
history = model.fit(
    dataset_train,
    batch_size=16,
    #steps_per_epoch=len(dataset_test)// batch_size,
    validation_data=dataset_validation,
    verbose=1,
    epochs=30,
)

Epoch 1/30


2023-06-06 21:39:23.811494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/12 [==========================>...] - ETA: 0s - loss: 1.6421 - accuracy: 0.1916

2023-06-06 21:39:25.236305: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 2s 112ms/step - loss: 1.6343 - accuracy: 0.2022 - val_loss: 1.5859 - val_accuracy: 0.2667
Epoch 2/30
12/12 [==============================] - 1s 72ms/step - loss: 1.5394 - accuracy: 0.3169 - val_loss: 1.5442 - val_accuracy: 0.3000
Epoch 3/30
12/12 [==============================] - 1s 66ms/step - loss: 1.3994 - accuracy: 0.3661 - val_loss: 1.5635 - val_accuracy: 0.1667
Epoch 4/30
12/12 [==============================] - 1s 69ms/step - loss: 1.3246 - accuracy: 0.4208 - val_loss: 1.7005 - val_accuracy: 0.2000
Epoch 5/30
12/12 [==============================] - 1s 66ms/step - loss: 1.2611 - accuracy: 0.5137 - val_loss: 1.6108 - val_accuracy: 0.2333
Epoch 6/30
12/12 [==============================] - 1s 64ms/step - loss: 1.1008 - accuracy: 0.5956 - val_loss: 1.4765 - val_accuracy: 0.3333
Epoch 7/30
12/12 [==============================] - 1s 67ms/step - loss: 0.8403 - accuracy: 0.7268 - val_loss: 1.4166 - val_accuracy: 0.4667
Epoch 8/30
12/12 [=====

In [62]:
score = model.evaluate(dataset_test, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

Validation loss: 5.155430793762207
Validation accuracy: 0.2222222238779068


In [63]:
label_test = dataset_test.map(lambda image, label:label)
label_test = np.concatenate([label.numpy() for _, label in dataset_test])

In [64]:
y_pred = model.predict(dataset_test)
y_pred = np.argmax(y_pred, axis=1)

1/1 [==============================] - 0s 168ms/step


2023-06-06 21:41:00.585484: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [65]:
con_mat = confusion_matrix(label_test, y_pred)
print(con_mat)

[[0 0 1 0 1]
 [0 0 2 0 0]
 [1 0 0 0 0]
 [1 0 1 0 0]
 [0 0 0 0 2]]
